In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact, FloatSlider, IntRangeSlider

from dask.distributed import Client, progress, wait
import dask.array as da
import dask, sys
import numpy as np
from numcodecs import Blosc

from semblance import semblance

/glb/data/cdis_projects/users/usnrej/attEngine/python-env/lib/python3.5/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import attengineshell as atte

In [3]:
client = Client(address='tcp://10.53.2.149:8786',  # Highlight this!!
                set_as_default=True, name='Client_1', 
                direct_to_workers=True)

In [4]:
inp_path = '/glb/data/cdis_projects/users/usnrej/attEngine/data/fourpoint/'
out_path = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/'

inp_file_zarr = inp_path+'/seis_fourpoint.zarr'
out_file_semblance_zarr = out_path+'/seis_fourpoint_out_sem.zarr'
out_file_envelope_zarr = out_path+'/seis_fourpoint_out_envelope._zarr'
out_file_instapha_zarr = out_path+'/seis_fourpoint_out_insta_phase.zarr'

In [5]:
semblance = atte.Semblance()
semblance.set_params(client, inp_file_zarr)
data_in, res_semblance  = semblance.run()

In [6]:
# fig=plt.figure(figsize=(16, 12), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams['figure.figsize'] = [16, 12]
def f(slice, xran, zran):
    xini = xran[0]
    xfin = xran[1]
    zini = zran[0]
    zfin = zran[1]
    d_input = data_in[xini:xfin,slice,zini:zfin].copy()
    d_sembl = res_semblance[xini:xfin,slice,zini:zfin].copy()
#     print(xini)
    if xfin <= xini or zfin <= zini:
        return
    plt.subplot(211)
    atte.plot_result_v01(np.rot90(d_input,1))
    plt.title('Input data')
    plt.subplot(212)
    atte.plot_result_v01(np.rot90(d_sembl,1))
    plt.title('Semblance')
#     plt.subplot(413)
#     atte.plot_result_v01(np.rot90(res_envel[xini:xfin,slice,zini:zfin],1))
#     plt.title('Envelope')
#     plt.subplot(414)
#     atte.plot_result_v01(np.rot90(res_instaphase[xini:xfin,slice,zini:zfin],1))
#     plt.title('Input data')
    
xmax, slicemax, zmax = data_in.shape

interact(f,slice=FloatSlider(value=812, min=0, max=slicemax-1, 
                             step=1, continuous_update=False),
         xran=IntRangeSlider(value=[675, 1395], min=0, max=xmax-1, 
                             step=1, continuous_update=False),
         zran=IntRangeSlider(value=[210, 829], min=0, max=zmax-1, 
                             step=1, continuous_update=False)
        )

interactive(children=(FloatSlider(value=812.0, continuous_update=False, description='slice', max=1956.0, step=…

<function __main__.f(slice, xran, zran)>

In [7]:
res_semblance.to_zarr(out_file_semblance_zarr, 
                      compute=True, 
                      overwrite=True, 
#                       compressor=None)
                      compressor=Blosc(cname='lz4', clevel=5, 
                                       shuffle=Blosc.SHUFFLE, blocksize=0))